In [1]:
'''
The purpose of this notebook is to load and clean the data.  
It should also test the data (e.g., by greatexpectations) to ensure that assumptions about the data are met, 
and store the cleaned and tested data.


'''

'\nThe purpose of this notebook is to load and clean the data.  \nIt should also test the data (e.g., by greatexpectations) to ensure that assumptions about the data are met, \nand store the cleaned and tested data.\n\n\n'

In [4]:
# Load libraries

from pandas import read_csv
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

#For great-expecations
import json
import great_expectations as ge
from great_expectations.profile import ColumnsExistProfiler
import great_expectations.jupyter_ux
from great_expectations.datasource.types import BatchKwargs
from datetime import datetime

In [3]:
# Load dataset
filename = "Restaurant_Scores_-_LIVES_Standard.csv"
names = ['business_id', 'business_name', 'business_address', 'business_city', 
         'business_state', 'business_postal_code', 'business_latitude', 'business_longitude', 
         'business_location', 'business_phone_number', 'inspection_id', 'inspection_date', 
         'inspection_score', 'inspection_type', 'violation_id', 'violation_description', 
         'risk_category'
        ]
dataset = read_csv(filename, names=names, low_memory = False)[1:] #removing the first row with names

#Take a first look at the data
print(dataset.head(5))

  business_id            business_name  business_address  business_city  \
1       69618  Fancy Wheatfield Bakery  1362 Stockton St  San Francisco   
2       97975               BREADBELLY   1408 Clement St  San Francisco   
3       69487   Hakkasan San Francisco       1 Kearny St  San Francisco   
4       91044    Chopsticks Restaurant   4615 Mission St  San Francisco   
5       85987                  Tselogs      552 Jones St  San Francisco   

  business_state business_postal_code business_latitude business_longitude  \
1             CA                94133               NaN                NaN   
2             CA                94118               NaN                NaN   
3             CA                94108               NaN                NaN   
4             CA                94112               NaN                NaN   
5             CA                94102               NaN                NaN   

  business_location business_phone_number   inspection_id inspection_date  \
1  

In [46]:
#Looking at the size of dataset
print("Number of rows: ", dataset.shape[0], '\nNumber of columns: ', dataset.shape[1])

Number of rows:  53973 
Number of columns:  17


In [11]:
#Looking at random slices of data

print(dataset[784:786])

    business_id business_name business_address  business_city business_state  \
785       86849  Lobsta Truck     Off The Grid  San Francisco             CA   
786       83423  Carbon Grill   852 Clement St  San Francisco             CA   

    business_postal_code business_latitude business_longitude  \
785                  NaN               NaN                NaN   
786                94118               NaN                NaN   

    business_location business_phone_number   inspection_id  \
785               NaN                   NaN  86849_20190822   
786               NaN          +14155759966  83423_20180612   

            inspection_date inspection_score        inspection_type  \
785  08/22/2019 12:00:00 AM              NaN  Reinspection/Followup   
786  06/12/2018 12:00:00 AM               72  Routine - Unscheduled   

              violation_id             violation_description  risk_category  
785                    NaN                               NaN            NaN  
786

## Great Expectation playground
This part is collection of my hello world expectations

In [9]:
#Get data context
context = ge.data_context.DataContext()

2020-03-07T00:02:11-0600 - INFO - Using project config: D:\Coding\ML study group\ML_workgroup\SFrestaurants\great_expectations\great_expectations.yml


In [11]:
#Get a pipeline run id
run_id = datetime.utcnow().strftime("%Y%m%dT%H%M%S.%fZ")
run_id

'20200307T060505.209572Z'

In [12]:
#List all CSV in the folder
ge.jupyter_ux.list_available_data_asset_names(context)

Inspecting your data sources. This may take a moment...


In [19]:
#Specify our data
data_asset_name = 'Restaurant_Scores_-_LIVES_Standard'
normalized_data_asset_name = context.normalize_data_asset_name(data_asset_name)
expectation_suite_name = 'warning'

In [21]:
# Basic expectation "Expect comlumn to exist"
try:
    context.get_expectation_suite(normalized_data_asset_name, expectation_suite_name)
except great_expectations.exceptions.DataContextError:
    context.create_expectation_suite(data_asset_name=normalized_data_asset_name, expectation_suite_name=expectation_suite_name, overwrite_existing=True);
    batch_kwargs = context.yield_batch_kwargs(data_asset_name)
    batch = context.get_batch(normalized_data_asset_name, expectation_suite_name, batch_kwargs)
    ColumnsExistProfiler().profile(batch)
    batch.save_expectation_suite()
    expectation_suite = context.get_expectation_suite(normalized_data_asset_name, expectation_suite_name)
    context.build_data_docs()

In [24]:
# Load a batch of data we want to validate
batch_kwargs = context.yield_batch_kwargs(data_asset_name)
batch = context.get_batch(
    data_asset_name = normalized_data_asset_name,
    expectation_suite_name = expectation_suite_name,
    batch_kwargs = batch_kwargs)
batch.head()

,business_id,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,business_location,business_phone_number,inspection_id,inspection_date,inspection_score,inspection_type,violation_id,violation_description,risk_category
0,69618,Fancy Wheatfield Bakery,1362 Stockton St,San Francisco,CA,94133,NaN,NaN,NaN,NaN,69618_20190304,03/04/2019 12:00:00 AM,NaN,Complaint,69618_20190304_103130,Inadequate sewage or wastewater disposal,Moderate Risk
1,97975,BREADBELLY,1408 Clement St,San Francisco,CA,94118,NaN,NaN,NaN,1.415724e+10,97975_20190725,07/25/2019 12:00:00 AM,96.0,Routine - Unscheduled,97975_20190725_103124,Inadequately cleaned or sanitized food contact...,Moderate Risk
2,69487,Hakkasan San Francisco,1 Kearny St,San Francisco,CA,94108,NaN,NaN,NaN,NaN,69487_20180418,04/18/2018 12:00:00 AM,88.0,Routine - Unscheduled,69487_20180418_103119,Inadequate and inaccessible handwashing facili...,Moderate Risk
3,91044,Chopsticks Restaurant,4615 Mission St,San Francisco,CA,94112,NaN,NaN,NaN,NaN,91044_20170818,08/18/2017 12:00:00 AM,NaN,Non-inspection site visit,NaN,NaN,NaN
4,85987,Tselogs,552 Jones St,San Francisco,CA,94102,NaN,NaN,NaN,NaN,85987_20180412,04/12/2018 12:00:00 AM,94.0,Routine - Unscheduled,85987_20180412_103132,Improper thawing methods,Moderate Risk


In [25]:
#Validate batch
validation_result = batch.validate(run_id=run_id)

if validation_result["success"]:
    print("This data meets all expectations for {}".format(str(data_asset_name)))
else:
    print("This data is not a valid batch of {}".format(str(data_asset_name)))

2020-03-07T00:16:35-0600 - INFO - 	17 expectation(s) included in expectation_suite.
This data meets all expectations for Restaurant_Scores_-_LIVES_Standard


In [26]:
#Review the validation results
print(json.dumps(validation_result, indent=4))

{
    "results": [
        {
            "success": true,
            "expectation_config": {
                "expectation_type": "expect_column_to_exist",
                "kwargs": {
                    "column": "business_id"
                }
            },
            "exception_info": {
                "raised_exception": false,
                "exception_message": null,
                "exception_traceback": null
            }
        },
        {
            "success": true,
            "expectation_config": {
                "expectation_type": "expect_column_to_exist",
                "kwargs": {
                    "column": "business_name"
                }
            },
            "exception_info": {
                "raised_exception": false,
                "exception_message": null,
                "exception_traceback": null
            }
        },
        {
            "success": true,
            "expectation_config": {
                "expectation_type": "expect_co

In [27]:
#Expectation "Expect values in the first column not to be null"
column_name = batch.get_table_columns()[0]
batch.expect_column_values_to_not_be_null(column_name)

{'success': True,
 'result': {'element_count': 53973,
  'unexpected_count': 0,
  'unexpected_percent': 0.0,
  'partial_unexpected_list': []}}

In [29]:
#Expectation "Expect values in the last column not to be null"
column_name = batch.get_table_columns()[16]
batch.expect_column_values_to_not_be_null(column_name)

{'success': False,
 'result': {'element_count': 53973,
  'unexpected_count': 13720,
  'unexpected_percent': 25.42011746614048,
  'partial_unexpected_list': []}}

In [32]:
# Expectation "Expect values in column 'risk_category' to be following values:
batch.expect_column_values_to_be_in_set('risk_category', ['Low Risk','Moderate Risk', 'High Risk', 'NaN'])

{'success': True,
 'result': {'element_count': 53973,
  'missing_count': 13720,
  'missing_percent': 25.42011746614048,
  'unexpected_count': 0,
  'unexpected_percent': 0.0,
  'unexpected_percent_nonmissing': 0.0,
  'partial_unexpected_list': []}}

In [33]:
# Expectation "Expect values in column 'inspection_type' to be following values:
batch.expect_column_values_to_be_in_set('inspection_type', 
                                        ['Complaint','Routine - Unscheduled', 'Non-inspection site visit', 'NaN'])

{'success': False,
 'result': {'element_count': 53973,
  'missing_count': 0,
  'missing_percent': 0.0,
  'unexpected_count': 11089,
  'unexpected_percent': 20.545457914142258,
  'unexpected_percent_nonmissing': 20.545457914142258,
  'partial_unexpected_list': ['New Ownership - Followup',
   'New Ownership',
   'Reinspection/Followup',
   'Reinspection/Followup',
   'Reinspection/Followup',
   'New Ownership - Followup',
   'New Ownership - Followup',
   'Reinspection/Followup',
   'New Ownership',
   'Reinspection/Followup',
   'Reinspection/Followup',
   'Reinspection/Followup',
   'Reinspection/Followup',
   'Structural Inspection',
   'Reinspection/Followup',
   'New Ownership - Followup',
   'New Ownership',
   'New Ownership',
   'New Ownership',
   'New Ownership']}}

In [34]:
#Review results
batch.get_expectation_suite()

2020-03-07T00:38:48-0600 - INFO - 	19 expectation(s) included in expectation_suite. Omitting 2 expectation(s) that failed when last run; set discard_failed_expectations=False to include them. result_format settings filtered.


{'data_asset_name': 'dataset/default/Restaurant_Scores_-_LIVES_Standard',
 'expectation_suite_name': 'warning',
 'meta': {'great_expectations.__version__': '0.8.8'},
 'expectations': [{'expectation_type': 'expect_column_to_exist',
   'kwargs': {'column': 'business_id'}},
  {'expectation_type': 'expect_column_to_exist',
   'kwargs': {'column': 'business_name'}},
  {'expectation_type': 'expect_column_to_exist',
   'kwargs': {'column': 'business_address'}},
  {'expectation_type': 'expect_column_to_exist',
   'kwargs': {'column': 'business_city'}},
  {'expectation_type': 'expect_column_to_exist',
   'kwargs': {'column': 'business_state'}},
  {'expectation_type': 'expect_column_to_exist',
   'kwargs': {'column': 'business_postal_code'}},
  {'expectation_type': 'expect_column_to_exist',
   'kwargs': {'column': 'business_latitude'}},
  {'expectation_type': 'expect_column_to_exist',
   'kwargs': {'column': 'business_longitude'}},
  {'expectation_type': 'expect_column_to_exist',
   'kwargs': {'

In [35]:
#Save results
batch.save_expectation_suite()

2020-03-07T00:39:48-0600 - INFO - 	19 expectation(s) included in expectation_suite. Omitting 2 expectation(s) that failed when last run; set discard_failed_expectations=False to include them. result_format settings filtered.


In [36]:
#Create and view Data Docs
context.build_data_docs()
context.open_data_docs()